# Setup Environment Variables

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# Load Dataset

In [2]:
import pandas as pd

In [3]:
df_summaries = pd.read_csv("./MovieSummaries/plot_summaries.txt", delimiter="\t", usecols=[0, 1], names=["id", "text"])

In [4]:
df_metadata = pd.read_csv("./MovieSummaries/movie.metadata.tsv", delimiter="\t", usecols=[0, 2, 3, 5], names=["id", "movie_title", "release_date", "runtime"])

In [5]:
df = pd.merge(df_metadata, df_summaries, how="inner", on="id")

In [6]:
df = df.fillna(-1)
df.text = df.text.astype(str)

# Convert Summaries to Embeddings

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
summary_embeddings = model.encode(df.text)

KeyboardInterrupt: 

In [47]:
import numpy as np
np.save("./MovieSummaries/summary_embeddings.npy", np.array(summary_embeddings))
np.save("./MovieSummaries/summary_id.npy", np.array(df.id))


# Load Embeddings

In [9]:
import numpy as np

# Load Embeddings
summary_embeddings = np.load("./MovieSummaries/summary_embeddings.npy")
ids = np.load("./MovieSummaries/summary_id.npy", allow_pickle=True)

In [10]:
ids

array([  975900,  9363483,   261236, ..., 34980460,   913762, 12476867])

# Upload to Pinecone

In [11]:
import pinecone
import os
import time

index_name = "cmu-movie-summaries"

In [12]:
api_key = os.getenv("PINECONE_API_KEY")
env = os.getenv("PINECONE_ENV") # found next to api key

In [13]:
pinecone.init(api_key=api_key, environment=env)

In [39]:
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

# create new index
pinecone.create_index(
    name=index_name,
    metric="cosine",
    dimension=summary_embeddings.shape[1]
)

while not pinecone.describe_index(index_name).status["ready"]:
    time.sleep(1)

In [40]:
index = pinecone.GRPCIndex(index_name)
index2 = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

In [42]:
import json

df_embedding = pd.DataFrame({
    "wiki_id": ids,
    "values": summary_embeddings.tolist(),
})

df_export = pd.merge(df_embedding, df, left_on="wiki_id", right_on="id", how="inner")

df_json_string = df_export[["release_date", "runtime", "text"]].to_json(orient="records")

df_export["metadata"] = json.loads(df_json_string)
df_export["id"] = df_export["movie_title"].str.encode(encoding="ascii", errors="ignore").astype(str)
df_export = df_export[df_export["id"].str.len() > 0] # some bulgarian movie cannot be converted to ascii

In [43]:
df_export[["id", "values", "metadata"]].head()

,id,values,metadata
0,Ghosts of Mars,"[-0.030300872400403023, 0.05613691732287407, -...","{'release_date': '2001-08-24', 'runtime': 98.0..."
1,White Of The Eye,"[-0.07510953396558762, 0.0031548410188406706, ...","{'release_date': '1987', 'runtime': 110.0, 'te..."
2,A Woman in Flames,"[-0.06540518254041672, 0.020126990973949432, -...","{'release_date': '1983', 'runtime': 106.0, 'te..."
3,The Sorcerer's Apprentice,"[-0.019909139722585678, 0.07503096014261246, -...","{'release_date': '2002', 'runtime': 86.0, 'tex..."
4,Little city,"[-0.07269977033138275, -0.07025031000375748, 0...","{'release_date': '1997-04-04', 'runtime': 93.0..."


In [44]:
index.upsert_from_dataframe(df_export[["id", "values", "metadata"]], batch_size=100)

sending upsert requests: 100%|██████████| 42203/42203 [00:28<00:00, 1467.43it/s]


upserted_count: 42203

In [46]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.38602,
 'namespaces': {'': {'vector_count': 38602}},
 'total_vector_count': 38602}

# Search

In [197]:
query = "cloud high above London"

In [198]:
query_embedding = model.encode(query)

In [199]:
res = index.query(
    vector=query_embedding,
    top_k=3,
)

In [200]:
res

{'matches': [{'id': 'Mary Poppins',
              'metadata': {},
              'score': 0.16821064,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': 'Henry V',
              'metadata': {},
              'score': 0.14026843,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': "The Sorcerer's Apprentice",
              'metadata': {},
              'score': 0.08186696,
              'sparse_values': {'indices': [], 'values': []},
              'values': []}],
 'namespace': ''}

In [ ]:
pinecone.delete_index(index_name)